# Webscraping
- 구인구직 사이트 웹스크래핑

> Indeed
- https://kr.indeed.com/%EC%B7%A8%EC%97%85?as_and=python&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=&radius=25&l=%EC%84%9C%EC%9A%B8&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch

> Stackoverflow


## Indeed

> 순서
- 1) 모듈설치
- 2) 가져올 페이지의 url요청 (get .text)
- 3) 원하는 html 파트 가져오기 (Beautiful Soup)
- 4) pagination 찾기
- 5-1) pagination 안의 모든 앵커 찾기
- 5-2) loop를 이용해 각 페이지의 "span" 모두 찾기
- 6) 불러올 페이지 넘버 지정해주기

> 참고
- div : division (section 분할한다고 생각)
- a : anchor (hyper link)

In [38]:
# 참고
# link(anchor)에서 string 추출해도 똑같은 결과가 나옴
# anchor가 있고, 이 요소 안에 다른 요소가 있고 그 요소에 string이 오직 하나 있다면 
# 그냥 anchor에서 string method 실행하면 됌
pages = []
for link in links[:-1]:
    pages.append(int(link.string))  # 그 안에 string만 가져오게 설정
                                    # int로 변경
max_page = pages[-1]

### Indeed function
- 위의 코드들과 나머지 코드들을 하나의 함수로 만듬

> 참고 코드 링크
- https://replit.com/@HJeongIn/Extracting-Indeed-Pages#indeed.py

> 참고
- 페이지 구조가 바뀌어서 다름
- #2.5 댓글들 보면 많은 사람들이 해결법 올려줌 (참고)

> Indeed의 [Jobs]
- Step1. URL 가져오기
- Step2. request 하기(to.서버)
- Step3. Jobs 가져오기(정보 추출,Parsing)
- 참고 - .string과 .text 차이 - https://www.inflearn.com/questions/3945

In [93]:
import requests
from bs4 import BeautifulSoup

LIMIT = 50
URL = "https://www.indeed.com/jobs?q=python&limit={}".format(LIMIT)

# indeed 페이지 중 가장 마지막 페이지 return
def get_last_page():
    result = requests.get(URL) # 실행시 에러 안 나오면 성공한 것임!
    soup = BeautifulSoup(result.text, "html.parser") # get을 통해 가져온 html document를 html.parser이용하여 쪼개기
    pagination = soup.find("div", {"class": "pagination"}) # class가 pagination인 html 가져와서 pagination에 저장
    links = pagination.find_all('a') # pagination html 속 모든! anchor(링크) 들을 리스트! 형태로 저장
    
    pages = [] # page 숫자들 저장할 List 
    for link in links[0:-1]: 
        pages.append(int(link.string)) 
    max_page = pages[-1] # 마지막 숫자 = 가장 큰 숫자
    return max_page

# job당 Title, Company, Location, ApplyLink로 정보 쪼개서 정리한 리스트 반환
def extract_job(html):
    title = html.select_one('.jobTitle>span').string #'new' 오류나는 title = html.find("h2", {"class":"jobTitle"}).find("span").string 대신 select_one 함수로!! 
    #<span> 속 title에 저장되어있는 job들 가져오기
    company = html.find("span", {"class":"companyName"}) # text내용을 가져와줄 .string #.string 안 해주면 html이 저장됨
    company_anchor = company.find("a") 
    #company 이름 중 링크(a)가 걸린 것도 아닌 것도 있음. 
    #company 정보가 <span>에 걸려 있는 경우와, <a>에 걸려 있는 찾아지는 경우가 나뉜다는 뜻
    if company_anchor is not None: # 링크(a)에 company 정보가 있는 경우
        company =str(company_anchor.string)  
    else:
        company = str(company.string) # 링크(a)에 없고 span에 company 정보가 있는 경우
    company = company.strip() # 빈 칸들 없애줌
    #print(title, company) 

    location = html.select_one("pre > div").text # 오류나는 html.find("div", {"class":"companyLocation"}).string 대신 select_one과 .text로!!!
    job_id = html.parent['data-jk'] # .parent 중요! 받아온 html의 부모 태그인 <a>에 [data-jk] 속성이 있음(사이트 업뎃 때문인듯)
    #print(job_id)# job_id는 지원링크의 id
    #print(location)
    #print(title)
    #print(result.status_code) # request가 last_page번 잘 되는지 확인해보는용
    return {
     'Title': title, 
     'Company': company, 
     'Location': location,
     'ApplyLnk': f"https://www.indeed.com/viewjob?jk={job_id}&from=web&vjs=3"
     }

# 해당 페이지의 모든 Job 추출
def extract_jobs(last_page):
    jobs = [] 
    for page in range(last_page): #page는 0부터 시작
        print("Scrapping Indeed Page : {}".format(page)) # 코드실행 진행여부 확인용
        result = requests.get("{0}&start={1}".format(URL, page*LIMIT)) #우리가 원하는 형태의 url로 request도 가능! 
        # 페이지 넘어갈 때마다! URL의 맨 뒤에 &limit={배열*50} 형태를 넣어준 상태에서 다음 페이지 request 해야함 #HTTP 통신상 한 URL씩 request해야 html 내용 가져올 수 있음
        soup = BeautifulSoup(result.text,"html.parser")
        results = soup.find_all("div", {"class":"slider_container"}) # 직업군 애용있는 html 가져오기

        for result in results:
            job = extract_job(result) # result은 html 저장중
            jobs.append(job)
    return jobs 

### main

In [94]:
def main():
    last_indeed_pages = get_last_page()
    indeed_jobs = extract_jobs(last_indeed_pages)
    print(indeed_jobs)

main()

Scrapping Indeed Page : 0
Scrapping Indeed Page : 1
Scrapping Indeed Page : 2
Scrapping Indeed Page : 3
Scrapping Indeed Page : 4
[{'Title': 'Senior Python Developer', 'Company': 'ShipHero', 'Location': 'Remote', 'ApplyLnk': 'https://www.indeed.com/viewjob?jk=1ef36fc836615b49&from=web&vjs=3'}, {'Title': 'Python Developer', 'Company': 'Shuup Commerce Inc', 'Location': 'Remote', 'ApplyLnk': 'https://www.indeed.com/viewjob?jk=f9534ae0091f5310&from=web&vjs=3'}, {'Title': 'Python Developer', 'Company': 'ESO', 'Location': 'Belcamp, MD', 'ApplyLnk': 'https://www.indeed.com/viewjob?jk=605b7c6764f51598&from=web&vjs=3'}, {'Title': 'Backend Python Engineer', 'Company': 'Play Magnus', 'Location': 'Remote', 'ApplyLnk': 'https://www.indeed.com/viewjob?jk=c27f1c0f6f92cb2c&from=web&vjs=3'}, {'Title': 'Software Engineer (PHP/Python)', 'Company': 'Catchpoint', 'Location': 'Remote', 'ApplyLnk': 'https://www.indeed.com/viewjob?jk=e06b448fecf5553d&from=web&vjs=3'}, {'Title': 'Machine Learning Engineer', 'C